In [2]:
import os
import math
import numpy as np

### Functinos

In [3]:
def feature_list(nx=64,ny=64,height=2,width=1,stride=1):
    f12_list = []
    f21_list = []
    for j in range(0,ny,stride):
        for i in range(0,nx, stride):
            ### 1 by 2 (x by y)
            if i+stride+width <= nx-1 and j+height <= ny-1:
                #shaded
                xs, ys  = i,j # upper left
                _xs,_ys = i+width, j+height# bottom right
                # unshaded
                xu, yu  = i+stride,j # upper left
                _xu,_yu = i+stride+width, j+height# bottom right
                # shaded + unshaded
                f12_list += [[[xs, ys],[_xs, _ys],[xu, yu],[_xu, _yu]]]
            ### 2 by 1 (x by y)
            if i+stride+width <= nx-1 and j+stride+width <= ny-1:
                #shaded
                xs, ys  = i,j # upper left
                _xs,_ys = i+height, j+width# bottom right
                # unshaded
                xu, yu  = i,j+stride # upper left
                _xu,_yu = i+height, j+stride+width# bottom right
                # shaded + unshaded
                f21_list += [[[xs, ys],[_xs, _ys],[xu, yu],[_xu, _yu]]]
    return f12_list+f21_list

In [4]:
def compute_feature(int_img_rep, feat_lst, feat_idx):
    """ int_img_rep: the N x 64 x 64 numpy matrix of the integral image representation
        feat_lst: list of features
        feat_idx: integer, index of a feature (in feat_lst)
        Returns: an N x 1 numpy matrix of the feature evaluations for each image
    """
    n, nx, ny = int_img_rep.shape
    feat_eval = []

    def _compute_intensity(int_img, ul, rb):
        """
        compute intensity
        x: upper left [ix, jx]
        y: bottom right [iy, jy]
        
        *(ix,iy)---*
        |          |
        |          |
        *----------*(jx,jy)
        """
        ix, iy = ul # upper left x,y
        jx, jy = rb #  bottom right x, y
        if ix-1 >= 0 and iy-1 >= 0  :
            intensity = int_img[jx,jy] + int_img[ix-1,iy-1] - int_img[ix-1,jy] - int_img[jx,iy-1]
        elif ix-1 >=0 and iy-1<0:
            intensity = int_img[jx,jy] - int_img[ix-1,jy]
        elif ix-1 < 0 and iy-1>=0:
            intensity = int_img[jx,jy]  - int_img[jx,iy-1]
        elif ix-1 < 0 and iy-1< 0:
            intensity = int_img[jx,jy] 
        return intensity
    
    for img in int_img_rep:
        """
        x: upper left in shaded
        y: bottom right in shaded
        z: upper left in unshaded
        s: bottom right in unshaded
        """
        x,y,z,s = feat_lst[feat_idx] 
        shaded   = _compute_intensity(img, x,y) 
        unshaded = _compute_intensity(img, z,s)
        feat_eval +=[shaded - unshaded]
    
    return np.asarray(feat_eval)

In [5]:
def compute_integral_image(imgs):
    """
    IN: imgs. np.ndarray, shape [#images, 64,64]
    """
    
    def _compute_int_img_unit(img):
        """
            compute integral image for one image 
        """
        nx, ny = img.shape
        int_img = np.ones((nx,ny))
        int_img[0,0] = img[0,0]
        for i in range(nx):
            for j in range(ny):
                if j > 0 and i > 0:
                    int_img[i,j] = int_img[i-1,j] + sum([ img[i,k] for k in range(j+1)])
                elif j==0 and i>0:
                    int_img[i,0] = int_img[i-1,0] + img[i,0]
                elif i==0 and j>0:
                    int_img[0,j] = int_img[0,j-1] + img[0,j]
        return int_img
    
    nimg, nx, ny = imgs.shape
    int_img_list = []
    for img in imgs:
        int_img_list += [_compute_int_img_unit(img).reshape(1,nx,ny)]
    int_imgs = np.concatenate(int_img_list, axis=0)
    return int_imgs

### Question 5.1

In [46]:
def opt_p_theta(int_img_rep, feat_lst, weights, feat_idx, y_true):
    """
    p = {+1,-1}
    theta = threshold
    """
    p, theta = 0,0
    #value of feature_idx on example 1-All
    _eval_list = [] # f(x_sigma)
    _eval_list += [compute_feature(int_img_rep=int_img_rep, feat_lst=feat_lst, feat_idx=feat_idx)]
    #permutation
    _eval_list.sort() # f(x_sigma1) < f(x_sigma2) ... < f(x_sigmaN)
    eval_list = np.squeeze(np.asarray(_eval_list))
    #print(eval_list.shape)
    
    #potential theta
    thetas = []
    for i in range(int_img_rep.shape[0]):
        theta_left = eval_list[i]
        if i+1 <= len(eval_list)-1:
            theta_right = eval_list[i+1]
        else:
            theta_right = theta_left
    
        # decide theta
        """
         f(x_sigma_j) <= theta <= f(x_sigma_j+1) j:= feat_idx
        """
        theta = theta_left + (theta_right-theta_left)/2
        thetas += [theta]
    
    #decide p
    ps = []
    plus_vec  = np.zeros(len(y_true))## length is y_true
    minus_vec = np.zeros(len(y_true))
    plus_vec[np.where(y_true == 1)] = 1 # binary vector
    minus_vec[np.where(y_true == -1)] = 1
    
    prod_plus = weights*plus_vec
    prod_minus = weights*minus_vec
    
    t_plus = np.sum(prod_plus)
    t_minus = np.sum(prod_minus)
    for i in range(int_img_rep.shape[0]):
        s_plus = np.sum(prod_plus[:i])
        s_minus = np.sum(prod_minus[:i])
        # Comment out lines are bug
        #epsilon = min((s_plus+(t_minus-s_minus)), (s_minus+(t_plus-s_plus)))
        #p = np.sign(epsilon)
        if s_plus+(t_minus-s_minus) < s_minus+(t_plus-s_plus):
            p = -1
        else:
            p = 1
        ps += [p]
    
    return np.asarray(ps), np.asarray(thetas)

In [45]:
dd=1
ee=2
min(dd,ee)

1

#### Practice case

In [7]:
nfile = 10
npatch=24
imgs = np.ones((nfile,npatch,npatch))
for l in range(nfile):
    for i in range(npatch):
        for j in range(npatch):
            imgs[l,i,j] = np.random.randint(0,255)

In [8]:
int_imgs = compute_integral_image(imgs)

In [9]:
w = np.ones((nfile))*(1/nfile) # weight
w

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [52]:
### make sudo label
y_true = np.ones((nfile))
for i in range(len(y_true)):
    if i < 9:
        y_true[i] = -1
    else:
        y_true[i] = 1
np.random.shuffle(y_true)
y_true

array([-1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.])

In [30]:
#### feature
flist = feature_list(nx=24,ny=24)
len(flist)

968

In [51]:
p, theta = opt_p_theta(int_imgs, flist, w, 8, y_true)
p, theta

(array([ 1,  1,  1,  1,  1,  1,  1, -1, -1, -1]),
 array([ 234.5,  135. ,  -57. , -139.5, -127. ,  -35.5,   97. ,  110.5,
        -130.5, -202. ]))

In [19]:
a=np.ones((5))
a[3] = 0
a[0] = 0
a

array([0., 1., 1., 0., 1.])

In [20]:
b = np.asarray([i for i in range(5)])
b

array([0, 1, 2, 3, 4])

In [21]:
a*b

array([0., 1., 2., 0., 4.])

In [41]:
np.random.randint(0,255)

187

### Question 5.2

In [17]:
def eval_learner(int_img_rep, feat_lst, feat_idx, p, theta):
    """    
    int_img_rep: the N x 64 x 64 numpy matrix of the integral image representation
    feat_lst: list of features
    feat_idx: integer, index of the feature for this weak learner
    p: +1 or -1, polarity
    theta: float, threshold
    Returns: N x 1 vector of label predictions for the given weak learner
    """
    ones = np.ones((int_img_rep.shape[0]))
    feature_vec = compute_feature(int_img_rep=int_img_rep, feat_lst=feat_lst, feat_idx=feat_idx)
    return np.sign(p*(feature_vec - theta))

In [53]:
eval_learner(int_imgs, flist, 4, p, theta)

array([-1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.])

In [54]:
y_true

array([-1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.])

### Question 5.3

In [20]:
def error_rate(int_img_rep, feat_lst, weights, feat_idx, p, theta, y_true):
    """
    int_img_rep: the N x 64 x 64 numpy matrix of the integral image representation
    feat_lst: list of features
    weights: an N x 1 matrix containing the weights for each datapoint
    feat_idx: integer, index of the feature for this weak learner
    p: +1 or -1, polarity
    theta: float, threshold
    y_true: N x 1 matrix of true labels
    Returns: the weighted error rate of this weak learner    
    """
    ### Weighted error of h_t
    """Classifier’s error rate.
        This is just the number of misclassifications over the training set 
        divided by the training set size.
    """
    ## correctness: number of correctly classified data
    eval_vec = eval_learner(int_img_rep, feat_lst, feat_idx, p, theta)
    error_vec = np.asarray(list(map(lambda x: 1 if x != 0 else 0, (y_true-eval_vec))))
    weighted_error = weights*error_vec
    weighted_error_rate = np.sum(weighted_error)/int_img_rep.shape[0]
    return weighted_error_rate

In [59]:
a = np.asarray([1,2,3,4])
b = np.asarray([1,0,3,0]) 
#for idx, _a in enumerate(a):
d = list(map(lambda x: 1 if x == 0 else 0, (a-b)))  
d

[1, 0, 1, 0]

In [21]:
erate = error_rate(int_imgs, flist, w, 4, p, theta, y_true)

In [22]:
erate

0.05

#### Question 5.4

In [48]:
def opt_weaklearner(int_img_rep, weights, feat_lst, y_true):
    """
    int_img_rep: the N x 64 x 64 numpy matrix of the integral image representation
    weights: N x 1 matrix of weights of each datapoint
    feat_lst: list of features
    Returns: the i (feature_idx), p, and theta values for the optimal weak learner
    """
    best_erate, best_feat_idx, best_p, best_theta =1.0e10, 0, 0, 0 # initialization
    for feat_idx in range(len(feat_lst)):
        p, theta = opt_p_theta(int_img_rep, feat_lst, weights, feat_idx, y_true)
        erate = error_rate(int_img_rep, feat_lst, weights, feat_idx, p, theta, y_true)
        if best_erate > erate:
            # update best error rate
            best_erate = erate 
            best_feat_idx = feat_idx
            best_p = p
            best_theta = theta
    return best_feat_idx, best_p, best_theta

In [55]:
opt_weaklearner(int_imgs, w, flist, y_true)

(227,
 array([ 1,  1,  1,  1,  1, -1, -1, -1, -1, -1]),
 array([  83. ,  259.5,  193. ,  108.5,  148. ,  119.5,   24. ,  -83. ,
        -100.5,  -60. ]))

In [63]:
best_feat_idx, best_p, best_theta = opt_weaklearner(int_imgs, w, flist, y_true)
best_feat_idx, best_p, best_theta

(227,
 array([ 1,  1,  1,  1,  1, -1, -1, -1, -1, -1]),
 array([  83. ,  259.5,  193. ,  108.5,  148. ,  119.5,   24. ,  -83. ,
        -100.5,  -60. ]))

In [65]:
erate = error_rate(int_imgs, flist, w, best_feat_idx, best_p, best_theta, y_true)
erate

0.02

In [68]:
y_pred = eval_learner(int_imgs, flist, best_feat_idx, best_p, best_theta)
y_pred

array([-1., -1.,  1., -1., -1., -1., -1., -1.,  1.,  0.])

### Question 6

In [80]:
def update_weights(weights, _error_rate, y_pred, y_true):
    """
    weights: N x 1 matrix containing the weights of each datapoint
    error_rate: the weighted error rate
    y_pred: N x 1 matrix of predicted labels
    y_true: N x 1 matrix of true labels
    Returns: N x 1 matrix of the updated weights
    """
    # scaler params
    alpha = 0.5*math.log((1-_error_rate)/_error_rate)
    zt = 2*math.sqrt(_error_rate*(1-_error_rate))
    new_weights = np.zeros((weights.shape))
    for i in range(len(y_true)):
        new_weights[i] = weights[i]*math.exp(-1*alpha*y_true[i]*y_pred[i])/zt
    return new_weights

In [81]:
new_w = update_weights(w, erate, y_pred, y_true)
new_w

array([0.05102041, 0.05102041, 2.5       , 0.05102041, 0.05102041,
       0.05102041, 0.05102041, 0.05102041, 0.05102041, 0.35714286])